<a href="https://colab.research.google.com/github/Fishing-oboro/NLP-learn/blob/main/TF_IDF%E3%81%AB%E3%81%A4%E3%81%84%E3%81%A6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# カウントベース
---
## BoW(Bag-of-Words)
---
bag of words(以下BoW)はカウントベースの手法に属しており、文書中に出現する単語の数を数え上げることからその名前が付けられている。
BoWでは文書中に出現する全ての単語にidを割り当て、各単語の出現数を記録する。そして出現数を対応する単語のidに割り当てることで文書ベクトルとする。
$$私[0]　は[1]　リンゴ[2]　と[3]　リンゴ[2]　を[4]　食べる[5]　。[6]→文書[1,1,2,1,1,1,1]$$


## TF-IDF
***
Bag-Of-Wordsのように文書内の単語を数えるだけでは、「私」や「これ」といった汎用的な言葉の影響力が大きくなってしまう。  
そのため、TF-IDFでは多くの文章に出現する単語の値が小さくなるように工夫がされている。
  
TF-IDFは**TF**(term frequency)と**IDF**(inverse document frequency)の積を意味する。

### TF (Term Frequency)
---
ある文章においてその単語が出現した割合を示す。一つの文章中に多く出現している単語ほど文章に関わるという考え方に基づいており、この値が大きいほどその単語が重要であるといえる。

　　
$$
tf(t,d) = \frac{n_{t,d}}{\sum_{s\in{d}}n_{s,d}}\\
(tf(t,d) : 文書d中の単語tのTF値\quad
n_{t,d} : 文書d中の単語tの出現数\quad
\sum_{s\in{d}}n_{s,d}: 文書d中の全単語の出現数の和)
$$

### IDF (Inverse Document Frequency)
---
ある文章においてその単語が出現した割合を示す。この値が大きいほどその単語が重要であるといえる。  
\
$$idf(t) = \log{\frac{N}{df(t)}}+1\\
(idf(t) : 単語tのIDF値\quad
N : 全文書数\quad
df(t): 単語tを含む文書数)
$$
\
後述のTfidfVectorizerではlog内の数が0にならないようにN及びdf(t)に1を加えた値が用いられている。  
\
$$idf(t) = \log{\frac{N+1}{df(t)+1}}+1\\
(idf(t) : 単語tのIDF値\quad
N : 全文書数\quad
df(t): 単語tを含む文書数)
$$

### cos類似度
---
単語や文章をベクトルで表すことで、以下の方程式に当てはめることができる。  


$$
\cos(\vec{a}, \vec{b})=\frac{\vec{a}・\vec{b}}{|\vec{a}||\vec{b}|}
=\frac{\sum{a_ib_i}}{\sqrt{\sum{a_i^2}}・\sqrt{\sum{b_i^2}}}
$$

cosの性質からcos(a,b)が1に近いほどa,bの性質が類似しており、0に近ければ類似していないといえる。

### 実装例
---
実際にTF-IDFを用いて、分散表現を作成する。
文章は[ferret](https://ferret-plus.com/)の記事5件とその関連記事1件を使用する。
まず必要なファイルをダウンロードした後にmecabを使い記事の形態素解析を行う。

In [ ]:
# mecabに必要なライブラリ取得
!apt-get install mecab libmecab-dev mecab-ipadic-utf8
!pip install mecab-python3
!ln -s /etc/mecabrc /usr/local/etc/mecabrc # シンボリックリンクの作成

In [ ]:
# テキストの取得
!wget https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/documents.txt
!wget https://raw.githubusercontent.com/Fishing-oboro/NLP-learn/main/index/statics/titles.txt

実装にはsklearnに搭載されているTfidfVectorizerとcosine_similarityを用いる。
これらを使うことで上に挙げた演算を容易に実装することができる。

In [ ]:
import numpy as np
import MeCab
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# input_text = open('./titles.txt', 'r').read()
input_text = open('./documents.txt', 'r').read()
documents = input_text.split("|")

def words(text):
    """
        文章から単語を抽出
    """
    out_words = []
    tagger = MeCab.Tagger('-Ochasen')
    tagger.parse('')
    node = tagger.parseToNode(text)

    while node:
        word_type = node.feature.split(",")[0]
        if word_type in ["名詞"]:
            out_words.append(node.surface)
        node = node.next
    return out_words


def vecs_array(documents):
    """
    各文章における重み付け
    """
    docs = np.array(documents)
    vectorizer = TfidfVectorizer(
        analyzer=words,
        stop_words='|',
        min_df=1,
        token_pattern='(?u)\\b\\w+\\b' #文字列長が1の単語を処理対象に含める
    )
    vecs = vectorizer.fit_transform(docs)
    print(vecs.shape) # (ベクトルの数, 次元数)
    return vecs.toarray()

# Cos類似度
input_title = open('./titles.txt', 'r').read()
tag = ["記事A", "記事B", "記事C", "記事D", "記事E", "記事F"]
titles = input_title.split("|")

vecs = vecs_array(documents)
print(vecs) # ベクトルの構造
cs_array = cosine_similarity(vecs, vecs)

for i, cs_item in enumerate(cs_array):
    print(tag[i] + ":" + titles[i])
    cs_dic = {}
    for j, cs in enumerate(cs_item):
        if round(cs - 1.0, 5) != 0: #同じ文書同士は省く
            cs_dic[tag[j]] = cs
    for k, v in sorted(cs_dic.items(), key=lambda x:x[1], reverse=True):
        print("\t" + k + " : " + str(v))